In [1]:
"""
Dataset extracted from Songsterr using selenium
Top 500 Drum Scores
Download as GP files
File conversion should be from GP to Midi or from GP to Music XML


"""

'\nDataset extracted from Songsterr using selenium\nTop 500 Drum Scores\nDownload as GP files\nFile conversion should be from GP to Midi or from GP to Music XML\n\n\n'

In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from concurrent.futures import ThreadPoolExecutor
from selenium.common.exceptions import InvalidSessionIdException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import asyncio
import time
import os

In [2]:
drum_score_folder = "/Users/jnaran/Desktop/SongsterrAPI/Drum_Scores"
sign_in_url = "https://www.songsterr.com/a/wa/signin"

In [3]:
#Define chrome_options
chrome_options = webdriver.ChromeOptions()
options = ['--headless', '--no-sandbox','--disable-dev-shm-usage']

for option in options:
  chrome_options.add_argument(option)

chrome_options.add_experimental_option("prefs",{
    "download.default_directory": drum_score_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
    
})

In [4]:
links_driver = webdriver.Chrome(options=chrome_options)
links_driver.get("https://songsterr.com/?inst=drum")

In [5]:
scroll_down = """
var searchWrap = document.getElementById('search-wrap');
searchWrap.scrollTop = searchWrap.scrollHeight;
"""

In [6]:
links = links_driver.find_elements(By.CSS_SELECTOR, "div[data-list='songs'] a[data-song]")

In [7]:
def get_links(links_driver,scroll_down):
    link_init = 0
    link_now = -1

    links = links_driver.find_elements(By.CSS_SELECTOR,"div[data-list='songs'] a[data-song]")  # initialize links
    
    while link_init != link_now:
        link_init = len(links)

        links_driver.execute_script(scroll_down)

        time.sleep(1)
        
        links = links_driver.find_elements(By.CSS_SELECTOR,"div[data-list='songs'] a[data-song]")

        link_now = len(links)

    return links

In [8]:
def urls(links):
    hrefs = []

    for link in links:
        href = link.get_attribute('href')
        hrefs.append(href)
        
    return hrefs


In [9]:
def return_links(links_driver,scroll_down):
    links = get_links(links_driver,scroll_down)
    hrefs = urls(links)
    return hrefs

In [10]:
songsterr_urls = return_links(links_driver,scroll_down)

In [11]:
len(songsterr_urls)

500

In [12]:
#Initializing driver to extract scores
score_driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(score_driver, 10)

In [13]:
def action_click(element,driver):
    actions = ActionChains(driver)
    actions.move_to_element(element)
    actions.click()
    actions.perform()

In [12]:
def login(login_driver):

    login_keys = ['drumioai@gmail.com','M@)5eVzXSU:p)zs']

    username = login_driver.find_element(By.XPATH, "//input[@type='email' and @name='email']")
    password = login_driver.find_element(By.XPATH, "//input[@type='password' and @name='password']")

    username.send_keys(login_keys[0])
    password.send_keys(login_keys[1])

    login_button = login_driver.find_element(By.CSS_SELECTOR, "#signin")
    login_button.click()


In [13]:
def login_check(score_driver,timeout=10):
    counter = 0
    while score_driver.current_url != "https://www.songsterr.com/":
        time.sleep(1)
        counter += 1
        if counter > timeout:
            raise Exception("Login timeout")
        else:
            pass
            return True
   

In [14]:
def get_gp_files(score_driver,songsterr_urls):
    
    score_driver = webdriver.Chrome(options=chrome_options)
    wait = We bDriverWait(score_driver, 10)
    score_driver.get(sign_in_url)
    login(score_driver)

    if login_check(score_driver) == True:
        print("Login successful")
        for url in songsterr_urls:
            score_driver.get(url)

            b1 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button#control-editor.Co02os")))
            b1.click()
            
            b2 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'rect.Bz2q3')))
            action_click(b2,score_driver)

            b3 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"button.z44l.z4xx[type='submit']")))
            b3.click()
            
            b4 = wait.until(EC.element_to_be_clickable((By.ID,"control-submitrevision")))
            b4.click()

            b5 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.Bta3n.Btaon")))
            try:
                    b5.click()
                    print('b5.click() executed successfully')
            except Exception as e:
                print(e)
                pass

In [2]:
from concurrent.futures import ThreadPoolExecutor

def main(score_driver, songsterr_urls):
    with ThreadPoolExecutor(max_workers=len(songsterr_urls)) as executor:
        # Use a lambda function to pass multiple arguments to get_gp_files
        executor.map(lambda url: get_gp_files(score_driver, url), songsterr_urls)

In [17]:
get_gp_files(score_driver, songsterr_urls)

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000104f3853c chromedriver + 3966268
1   chromedriver                        0x0000000104f30ac8 chromedriver + 3934920
2   chromedriver                        0x0000000104bb3da0 chromedriver + 277920
3   chromedriver                        0x0000000104bf6394 chromedriver + 549780
4   chromedriver                        0x0000000104c2ebf0 chromedriver + 781296
5   chromedriver                        0x0000000104beafb0 chromedriver + 503728
6   chromedriver                        0x0000000104beba28 chromedriver + 506408
7   chromedriver                        0x0000000104efd724 chromedriver + 3725092
8   chromedriver                        0x0000000104f01c18 chromedriver + 3742744
9   chromedriver                        0x0000000104ee620c chromedriver + 3629580
10  chromedriver                        0x0000000104f02714 chromedriver + 3745556
11  chromedriver                        0x0000000104ed9584 chro

In [48]:
# Test environmemnt for login

driver = webdriver.Chrome()
driver.get(sign_in_url)

In [49]:
login(driver)